In [1]:
from importlib import reload
from tqdm.notebook import tqdm
import librosa
import numpy as np
import torch
from torch import nn

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


## data loading

In [4]:
import src.data
reload(src.data)
from src.data import get_loaders, AudioDataset

dataset = AudioDataset(data_path='data/', split_gender=False)
train_loader, val_loader = get_loaders(dataset, bs=16, val_frac=0.1)
x, y = dataset[100]
x.shape, y

train samples: 1296 val samples: 144


(torch.Size([1, 124]), 2)

In [7]:
for batch in train_loader:
    x, y = batch
    print(x.shape, y.shape)
    break

torch.Size([16, 1, 124]) torch.Size([16])


In [ ]:
# from joblib import Parallel, delayed


# func = lambda idx: dataset[idx][0].shape[1]
# res = Parallel(n_jobs=4)(delayed(func)(file) for file in tqdm(range(len(dataset))))
# min(res), max(res)

## model

In [13]:
import src.model
reload(src.model)
from src.model import EmotionRecognizer

model = EmotionRecognizer(in_feat=dataset.num_features, num_classes=dataset.num_classes, p_dropout=0.2, lr=1e-4)
print(model)
print('num params:', model.count_parameters())

EmotionRecognizer(
  (conv): Sequential(
    (0): Conv1d(1, 32, kernel_size=(9,), stride=(2,), padding=(4,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv1d(64, 128, kernel_size=(5,), stride=(2,), padding=(2,))
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
  )
  (head): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=8, bias=True)
  )
  (criterion): CrossEntropyLoss()
  (accuracy): Accuracy()
)
num params: 72648


In [10]:
h = model(x)
h.shape

torch.Size([16, 8])

## train

In [20]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

# logger = TensorBoardLogger(save_dir='logs/',name='cnn_logs')
trainer = Trainer(max_epochs=30, logger=logger)

GPU available: False, used: False
I0212 23:33:03.465293 4544525760 distributed.py:49] GPU available: False, used: False
TPU available: False, using: 0 TPU cores
I0212 23:33:03.468720 4544525760 distributed.py:49] TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, train_loader, val_loader)


  | Name      | Type             | Params
-----------------------------------------------
0 | conv      | Sequential       | 56.3 K
1 | head      | Sequential       | 16.4 K
2 | criterion | CrossEntropyLoss | 0     
3 | accuracy  | Accuracy         | 0     
I0212 23:33:04.060252 4544525760 lightning.py:1459] 
  | Name      | Type             | Params
-----------------------------------------------
0 | conv      | Sequential       | 56.3 K
1 | head      | Sequential       | 16.4 K
2 | criterion | CrossEntropyLoss | 0     
3 | accuracy  | Accuracy         | 0     


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]